In [37]:
%matplotlib qt 

import numpy as np
import pandas as pd
import random
import time
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from imblearn.metrics import specificity_score

from scipy.stats import iqr
from scipy.stats import t

In [2]:
# import data

data = pd.read_csv('bm_combat_spectral_changed.csv', index_col=[0])

In [8]:
# harmonized 
data_comb = data.drop(data.iloc[:, 209:], axis=1)
data_comb

,center,group,age,gender,batch,label,AF3__d_combat,AF3__t_combat,AF3__st_combat,AF3__prealpha_combat,...,T7__a_combat,T7__b_combat,T7__atr_combat,T8__d_combat,T8__t_combat,T8__st_combat,T8__prealpha_combat,T8__a_combat,T8__b_combat,T8__atr_combat
c_hc1,california,hc,54,f,4,0,-1.023886,-1.037227,-1.712651,-1.190235,...,-0.503588,-0.302289,0.633863,-0.894442,-0.958995,-1.484516,-1.181637,-0.562981,-0.400004,0.417223
c_hc10,california,hc,59,f,4,0,-1.070873,-1.116702,-1.787769,-1.272548,...,-0.541652,-0.240732,0.496244,-1.161725,-1.097693,-1.744250,-1.245003,-0.581887,-0.218555,0.489160
c_hc18,california,hc,57,m,4,0,-0.999397,-0.977795,-1.628822,-1.136613,...,-0.290853,-0.660351,0.825784,-1.018688,-1.054559,-1.702455,-1.197703,-0.333537,-0.543787,0.698254
c_hc2,california,hc,50,f,4,0,-0.783938,-0.847593,-1.532702,-0.988887,...,-0.289275,-0.333501,0.900043,-0.885177,-0.970096,-1.592595,-1.137546,-0.462240,-0.476372,0.527589
c_hc20,california,hc,68,m,4,0,-0.733530,-0.602222,-1.033768,-0.862262,...,-0.743405,-0.360869,0.247448,-0.759434,-0.798566,-1.237258,-1.044402,-0.727774,-0.487117,0.126307
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
m_70,medellin,pd,63,f,1,1,-1.138536,-0.942143,-1.575136,-1.084559,...,-0.266917,-0.535315,0.786154,-1.121632,-1.026262,-1.653943,-1.168271,-0.242664,-0.652156,0.783598
m_71,medellin,pd,62,m,1,1,-0.773759,-0.696675,-1.234651,-0.872914,...,-0.809904,-0.266638,0.109196,-0.684435,-0.740871,-1.293089,-0.912333,-0.792552,-0.474612,-0.051681
m_72,medellin,pd,55,f,1,1,-0.699611,-0.843186,-1.264696,-1.085802,...,-0.442747,-0.606066,0.351313,-0.610813,-0.796654,-1.154333,-1.085144,-0.523925,-0.639380,0.272728
m_8,medellin,hc,75,m,1,0,-0.822632,-0.785273,-1.680908,-0.854227,...,-0.484869,-0.866680,0.462081,-0.768496,-0.882158,-1.766482,-0.953585,-0.382808,-0.712498,0.499351


In [9]:
np.random.seed(0)
random.seed(0)

X = data_comb
y = data_comb[['center','label']]

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    train_size=0.7,
                                                    random_state=1,
                                                    stratify=X[['center','group']])

In [12]:
center_names = ['all','california','finland','iowa','medellin']

X_train_sets = {} 
y_train_sets = {}
X_test_sets = {}
y_test_sets = {}


for name in center_names:
    
    if name == 'all':
        X_train_sets.update({name: X_train.drop(['center', 'group', 'age', 'gender', 'batch', 'label'], axis=1)})
        X_test_sets.update({name: X_test.drop(['center', 'group', 'age', 'gender', 'batch', 'label'], axis=1)})
        y_train_sets.update({name: y_train.drop(['center'], axis = 1)})
        y_test_sets.update({name: y_test.drop(['center'], axis = 1)})
    
    else: 
        X_train_set = X_train.loc[X_train['center'] == name]
        X_train_set = X_train_set.drop(['center', 'group', 'age', 'gender', 'batch', 'label'], axis=1)
        X_train_sets.update({name: X_train_set})

        y_train_set = y_train.loc[X_train['center'] == name]
        y_train_set = y_train_set.drop(['center'], axis=1)
        y_train_sets.update({name: y_train_set})

        X_test_set = X_test.loc[X_test['center'] == name]
        X_test_set = X_test_set.drop(['center', 'group', 'age', 'gender', 'batch', 'label'], axis=1)
        X_test_sets.update({name: X_test_set})

        y_test_set = y_test.loc[X_test['center'] == name]
        y_test_set = y_test_set.drop(['center'], axis=1)
        y_test_sets.update({name: y_test_set})

In [17]:
# Initializing Classifiers

clf1 = LogisticRegression(multi_class='multinomial',
                          solver='newton-cg',
                          random_state=1)

clf2 = KNeighborsClassifier(algorithm='ball_tree',
                            leaf_size=50)

clf3 = DecisionTreeClassifier(random_state=1)

clf4 = SVC(random_state=1)

# Building the pipelines

pipe1 = Pipeline([('clf1', clf1)])

pipe2 = Pipeline([('clf2', clf2)])

pipe4 = Pipeline([('clf4', clf4)])


# Setting up the parameter grids
param_grid1 = [{'clf1__penalty': ['l2'],
                'clf1__C': np.power(10., np.arange(-4, 4))}]

param_grid2 = [{'clf2__n_neighbors': list(range(1, 10)),
                'clf2__p': [1, 2]}]

param_grid3 = [{'max_depth': list(range(1, 10)) + [None],
                'criterion': ['gini', 'entropy']}]


param_grid4 = [
               {
                'clf4__kernel': ['rbf'],
                'clf4__C': np.power(10., np.arange(-4, 4)),
                'clf4__gamma': np.power(10., np.arange(-5, 0))},
               {
                'clf4__kernel': ['linear'],
                'clf4__C': np.power(10., np.arange(-4, 4))}
              ]

In [18]:
# Setting up multiple GridSearchCV objects, 1 for each algorithm

gridcvs = {}
    
for pgrid, est, name in zip((param_grid1, param_grid4),
                            (pipe1, pipe4),
                            ('Softmax', 'SVM')):
    
    gcv = GridSearchCV(estimator=est,
                       param_grid=pgrid,
                       scoring='accuracy',
                       n_jobs=1,
                       cv=5,
                       verbose=0,
                       refit=True)
    gridcvs[name] = gcv

# WITH FEATURE SELECTION: RENT

In [20]:
# FEATURE SELECTION MODEL

import sys
sys.path.append('../src')
from RENT import RENT, stability

import warnings
warnings.filterwarnings("ignore")

# Define a range of regularisation parameters C for elastic net. A minimum of at least one value is required.
my_C_params = [0.1, 1, 10]

# Define a reange of l1-ratios for elastic net.  A minimum of at least one value is required.
my_l1_ratios = [0, 0.1, 0.25, 0.5, 0.75, 0.9, 1]

In [21]:
list_of_k = np.linspace(0, 1, num=101)
list_of_k

array([0.  , 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 ,
       0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21,
       0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32,
       0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43,
       0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54,
       0.55, 0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65,
       0.66, 0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76,
       0.77, 0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87,
       0.88, 0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98,
       0.99, 1.  ])

In [25]:
metrics = ['accuracy','recall','specificity','precision','F1','auc']
cv_scores = {k: {center_name: {name: {metric: [] for metric in metrics} for name, gs_est in gridcvs.items()} for center_name in center_names} for k in list_of_k}
cv_scores_without_zeros = {k: {center_name: {name: {metric: [] for metric in metrics} for name, gs_est in gridcvs.items()} for center_name in center_names} for k in list_of_k}

skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

features_ids = {k: {center_name: [] for center_name in center_names} for k in list_of_k}
features_names = {k: {center_name: [] for center_name in center_names} for k in list_of_k}

for k in list_of_k:

    for center_name in center_names:
        
        if center_name != center_names[0]: continue

        print('Cross-validation + feature selection for number of features of ' + str(k) + ' for ' + center_name.upper())

        # The outer loop for algorithm selection
        c = 1
        for outer_train_idx, outer_valid_idx in skfold.split(X_train_sets[center_name],y_train_sets[center_name]):

            # feature selection: starts here
    
            model = RENT.RENT_Classification(data=X_train_sets[center_name].iloc[outer_train_idx.tolist()], 
                                                 target=y_train_sets[center_name].iloc[outer_train_idx.tolist()].values.ravel(), 
                                                 feat_names=X_train_sets[center_name].columns, 
                                                 C=my_C_params, 
                                                 l1_ratios=my_l1_ratios,
                                                 autoEnetParSel=True,
                                                 BIC=False,
                                                 poly='OFF',
                                                 testsize_range=(0.25,0.25),
                                                 scoring='accuracy',
                                                 classifier='logreg',
                                                 K=100,
                                                 random_state=0,
                                                 verbose=1)

            model.train()
            model.get_enet_params()
            selected_features = model.select_features(tau_1_cutoff=k, tau_2_cutoff=k, tau_3_cutoff=k)
            features = X_train_sets[center_name].columns[selected_features].tolist()

            features_ids[k][center_name].append(selected_features)
            features_names[k][center_name].append(features)

            # feature selection: ends here

            for name, gs_est in sorted(gridcvs.items()):

                # The inner loop for hyperparameter tuning

                if len(features) != 0:

                    gs_est.fit(X_train_sets[center_name].iloc[outer_train_idx.tolist()].iloc[:,selected_features], y_train_sets[center_name].iloc[outer_train_idx.tolist()].values.ravel())
                    y_pred = gs_est.predict(X_train_sets[center_name].iloc[outer_valid_idx.tolist()].iloc[:,selected_features])

                    for metric in metrics: 

                        if metric == 'accuracy':
                            calc_metric = accuracy_score(y_true=y_train_sets[center_name].iloc[outer_valid_idx.tolist()], y_pred=y_pred)
                            print(' | inner ACC %.2f%% | outer ACC %.2f%%' %
                                  (gs_est.best_score_ * 100, calc_metric * 100))
                            cv_scores[k][center_name][name][metric].append(calc_metric)
                            if calc_metric != 0:
                                cv_scores_without_zeros[k][center_name][name][metric].append(calc_metric)
                        elif metric == 'recall':
                            calc_metric = recall_score(y_true=y_train_sets[center_name].iloc[outer_valid_idx.tolist()], y_pred=y_pred)
                            cv_scores[k][center_name][name][metric].append(calc_metric)
                            if calc_metric != 0:
                                cv_scores_without_zeros[k][center_name][name][metric].append(calc_metric)
                        elif metric == 'specificity':
                            calc_metric = specificity_score(y_true=y_train_sets[center_name].iloc[outer_valid_idx.tolist()], y_pred=y_pred)
                            cv_scores[k][center_name][name][metric].append(calc_metric)
                            if calc_metric != 0:
                                cv_scores_without_zeros[k][center_name][name][metric].append(calc_metric)
                        elif metric == 'precision':
                            calc_metric = precision_score(y_true=y_train_sets[center_name].iloc[outer_valid_idx.tolist()], y_pred=y_pred)
                            cv_scores[k][center_name][name][metric].append(calc_metric)
                            if calc_metric != 0:
                                cv_scores_without_zeros[k][center_name][name][metric].append(calc_metric)
                        elif metric == 'F1':
                            calc_metric = f1_score(y_true=y_train_sets[center_name].iloc[outer_valid_idx.tolist()], y_pred=y_pred)
                            cv_scores[k][center_name][name][metric].append(calc_metric)
                            if calc_metric != 0:
                                cv_scores_without_zeros[k][center_name][name][metric].append(calc_metric)
                        else:
                            calc_metric = roc_auc_score(y_true=y_train_sets[center_name].iloc[outer_valid_idx.tolist()], y_score=y_pred, average = 'macro')
                            cv_scores[k][center_name][name][metric].append(calc_metric)
                            if calc_metric != 0:
                                cv_scores_without_zeros[k][center_name][name][metric].append(calc_metric)
                else:

                    print('!!!NO FEATURES WERE SELECTED!!!')

                    gs_est.fit(X_train_sets[center_name].iloc[outer_train_idx.tolist()], y_train_sets[center_name].iloc[outer_train_idx.tolist()].values.ravel())
                    y_pred = gs_est.predict(X_train_sets[center_name].iloc[outer_valid_idx.tolist()])

                    for metric in metrics: 

                        if metric == 'accuracy':
                            calc_metric = accuracy_score(y_true=y_train_sets[center_name].iloc[outer_valid_idx.tolist()], y_pred=y_pred)
                            print(' | inner ACC %.2f%% | outer ACC %.2f%%' %
                                  (gs_est.best_score_ * 100, calc_metric * 100))
                            cv_scores[k][center_name][name][metric].append(calc_metric)
                            if calc_metric != 0:
                                cv_scores_without_zeros[k][center_name][name][metric].append(calc_metric)
                        elif metric == 'recall':
                            calc_metric = recall_score(y_true=y_train_sets[center_name].iloc[outer_valid_idx.tolist()], y_pred=y_pred)
                            cv_scores[k][center_name][name][metric].append(calc_metric)
                            if calc_metric != 0:
                                cv_scores_without_zeros[k][center_name][name][metric].append(calc_metric)
                        elif metric == 'specificity':
                            calc_metric = specificity_score(y_true=y_train_sets[center_name].iloc[outer_valid_idx.tolist()], y_pred=y_pred)
                            cv_scores[k][center_name][name][metric].append(calc_metric)
                            if calc_metric != 0:
                                cv_scores_without_zeros[k][center_name][name][metric].append(calc_metric)
                        elif metric == 'precision':
                            calc_metric = precision_score(y_true=y_train_sets[center_name].iloc[outer_valid_idx.tolist()], y_pred=y_pred)
                            cv_scores[k][center_name][name][metric].append(calc_metric)
                            if calc_metric != 0:
                                cv_scores_without_zeros[k][center_name][name][metric].append(calc_metric)
                        elif metric == 'F1':
                            calc_metric = f1_score(y_true=y_train_sets[center_name].iloc[outer_valid_idx.tolist()], y_pred=y_pred)
                            cv_scores[k][center_name][name][metric].append(calc_metric)
                            if calc_metric != 0:
                                cv_scores_without_zeros[k][center_name][name][metric].append(calc_metric)
                        else:
                            calc_metric = roc_auc_score(y_true=y_train_sets[center_name].iloc[outer_valid_idx.tolist()], y_score=y_pred, average = 'macro')
                            cv_scores[k][center_name][name][metric].append(calc_metric)
                            if calc_metric != 0:
                                cv_scores_without_zeros[k][center_name][name][metric].append(calc_metric)


            c += 1

Cross-validation + feature selection for number of features of 0.0 for ALL
data dimension: (94, 203)  data type: <class 'pandas.core.frame.DataFrame'>
target dimension: (94,)
regularization parameters C: [0.1, 1, 10]
elastic net l1_ratios: [0, 0.1, 0.25, 0.5, 0.75, 0.9, 1]
poly: OFF
number of models in ensemble: 100
random state: 0
verbose: 1


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   7 out of   7 | elapsed:    6.5s finished


classifier: logreg
scoring: accuracy
data dimension: (94, 203)  data type: <class 'pandas.core.frame.DataFrame'>
target dimension: (94,)
regularization parameters C: [0.1, 1, 10]
elastic net l1_ratios: [0, 0.1, 0.25, 0.5, 0.75, 0.9, 1]
poly: OFF
number of models in ensemble: 100
random state: 0
verbose: 1


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   7 out of   7 | elapsed:   10.8s finished


classifier: logreg
scoring: accuracy
data dimension: (94, 203)  data type: <class 'pandas.core.frame.DataFrame'>
target dimension: (94,)
regularization parameters C: [0.1, 1, 10]
elastic net l1_ratios: [0, 0.1, 0.25, 0.5, 0.75, 0.9, 1]
poly: OFF
number of models in ensemble: 100
random state: 0
verbose: 1


...

classifier: logreg
scoring: accuracy
data dimension: (95, 203)  data type: <class 'pandas.core.frame.DataFrame'>
target dimension: (95,)
regularization parameters C: [0.1, 1, 10]
elastic net l1_ratios: [0, 0.1, 0.25, 0.5, 0.75, 0.9, 1]
poly: OFF
number of models in ensemble: 100
random state: 0
verbose: 1


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   7 out of   7 | elapsed:    7.4s finished


classifier: logreg
scoring: accuracy
!!!NO FEATURES WERE SELECTED!!!
!!!NO FEATURES WERE SELECTED!!!
data dimension: (95, 203)  data type: <class 'pandas.core.frame.DataFrame'>
target dimension: (95,)
regularization parameters C: [0.1, 1, 10]
elastic net l1_ratios: [0, 0.1, 0.25, 0.5, 0.75, 0.9, 1]
poly: OFF
number of models in ensemble: 100
random state: 0
verbose: 1


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   7 out of   7 | elapsed:    7.4s finished


classifier: logreg
scoring: accuracy
!!!NO FEATURES WERE SELECTED!!!
!!!NO FEATURES WERE SELECTED!!!


In [26]:
# Looking at the results


mean_std_cv_scores = {k: {center_name: {name: {metric: {'mean': None, 'SD': None} for metric in metrics} for name, gs_est in gridcvs.items()} for center_name in center_names} for k in list_of_k}
mean_std_cv_scores_without_zeros = {k: {center_name: {name: {metric: {'mean': None, 'SD': None} for metric in metrics} for name, gs_est in gridcvs.items()} for center_name in center_names} for k in list_of_k}
median_iqr_cv_scores = {k: {center_name: {name: {metric: {'median': None, 'IQR': None} for metric in metrics} for name, gs_est in gridcvs.items()} for center_name in center_names} for k in list_of_k}
median_iqr_cv_scores_without_zeros = {k: {center_name: {name: {metric: {'median': None, 'IQR': None} for metric in metrics} for name, gs_est in gridcvs.items()} for center_name in center_names} for k in list_of_k}

for k in list_of_k:
    
    for center_name in center_names:
    
        for name, gs_est in sorted(gridcvs.items()):
            print('Results for ' + center_name.upper() + ' ' + str(k) + ' ' + name.upper())
            for metric in metrics: 
                if metric != 'auc':
                    print('%-8s | outer CV ' + metric + ' %.2f%% +\- %.3f' % (
                      100 * np.mean(cv_scores[k][center_name][name][metric]), 100 * np.std(cv_scores[k][center_name][name][metric])))
                    mean_std_cv_scores[k][center_name][name][metric]['mean'] = np.mean(cv_scores[k][center_name][name][metric])
                    mean_std_cv_scores_without_zeros[k][center_name][name][metric]['mean'] = np.mean(cv_scores_without_zeros[k][center_name][name][metric])
                    mean_std_cv_scores[k][center_name][name][metric]['SD'] = np.std(cv_scores[k][center_name][name][metric])
                    mean_std_cv_scores_without_zeros[k][center_name][name][metric]['SD'] = np.std(cv_scores_without_zeros[k][center_name][name][metric])
                    
                    median_iqr_cv_scores[k][center_name][name][metric]['median'] = 100*np.median(cv_scores[k][center_name][name][metric])
                    median_iqr_cv_scores_without_zeros[k][center_name][name][metric]['median'] = 100*np.median(cv_scores_without_zeros[k][center_name][name][metric])
                    median_iqr_cv_scores[k][center_name][name][metric]['IQR'] = 100*iqr(cv_scores[k][center_name][name][metric])
                    median_iqr_cv_scores_without_zeros[k][center_name][name][metric]['IQR'] = 100*iqr(cv_scores_without_zeros[k][center_name][name][metric])
                else: 
                    print('%-8s | outer CV ' + metric + ' %.2f +\- %.3f' % (
                      np.mean(cv_scores[k][center_name][name][metric]), np.std(cv_scores[k][center_name][name][metric])))
                    mean_std_cv_scores[k][center_name][name][metric]['mean'] = np.mean(cv_scores[k][center_name][name][metric])
                    mean_std_cv_scores_without_zeros[k][center_name][name][metric]['mean'] = np.mean(cv_scores_without_zeros[k][center_name][name][metric])
                    mean_std_cv_scores[k][center_name][name][metric]['SD'] = np.std(cv_scores[k][center_name][name][metric])
                    mean_std_cv_scores_without_zeros[k][center_name][name][metric]['SD'] = np.std(cv_scores_without_zeros[k][center_name][name][metric])
                    
                    median_iqr_cv_scores[k][center_name][name][metric]['median'] = 100*np.median(cv_scores[k][center_name][name][metric])
                    median_iqr_cv_scores_without_zeros[k][center_name][name][metric]['median'] = 100*np.median(cv_scores_without_zeros[k][center_name][name][metric])
                    median_iqr_cv_scores[k][center_name][name][metric]['IQR'] = 100*iqr(cv_scores[k][center_name][name][metric])
                    median_iqr_cv_scores_without_zeros[k][center_name][name][metric]['IQR'] = 100*iqr(cv_scores_without_zeros[k][center_name][name][metric])
        print('*********************')
#print('\nSoftmax Best parameters', gridcvs['Softmax'].best_params_)
#print('\nKNN Best parameters', gridcvs['KNN'].best_params_)
#print('\nDTree Best parameters', gridcvs['DTree'].best_params_)
#print('\nSVM Best parameters', gridcvs['SVM'].best_params_)

Results for ALL 0.0 SVM
%-8s | outer CV accuracy 66.23% +\- 13.782
%-8s | outer CV recall 61.52% +\- 28.026
%-8s | outer CV specificity 71.52% +\- 13.318
%-8s | outer CV precision 66.22% +\- 17.776
%-8s | outer CV F1 61.51% +\- 21.613
%-8s | outer CV auc 0.67 +\- 0.139
Results for ALL 0.0 SOFTMAX
%-8s | outer CV accuracy 67.93% +\- 12.312
%-8s | outer CV recall 64.70% +\- 19.730
%-8s | outer CV specificity 71.52% +\- 15.262
%-8s | outer CV precision 69.90% +\- 14.973
%-8s | outer CV F1 65.90% +\- 15.149
%-8s | outer CV auc 0.68 +\- 0.123
*********************
Results for CALIFORNIA 0.0 SVM
%-8s | outer CV accuracy nan% +\- nan
%-8s | outer CV recall nan% +\- nan
%-8s | outer CV specificity nan% +\- nan
%-8s | outer CV precision nan% +\- nan
%-8s | outer CV F1 nan% +\- nan
%-8s | outer CV auc nan +\- nan
Results for CALIFORNIA 0.0 SOFTMAX
%-8s | outer CV accuracy nan% +\- nan
%-8s | outer CV recall nan% +\- nan
%-8s | outer CV specificity nan% +\- nan
%-8s | outer CV precision nan% +\- 

%-8s | outer CV specificity 78.33% +\- 12.472
%-8s | outer CV precision 72.45% +\- 18.576
%-8s | outer CV F1 58.80% +\- 16.116
%-8s | outer CV auc 0.66 +\- 0.107
Results for ALL 0.2 SOFTMAX
%-8s | outer CV accuracy 69.64% +\- 12.211
%-8s | outer CV recall 63.18% +\- 21.918
%-8s | outer CV specificity 76.67% +\- 13.333
%-8s | outer CV precision 72.70% +\- 18.454
%-8s | outer CV F1 66.00% +\- 17.442
%-8s | outer CV auc 0.70 +\- 0.124
*********************
Results for CALIFORNIA 0.2 SVM
%-8s | outer CV accuracy nan% +\- nan
%-8s | outer CV recall nan% +\- nan
%-8s | outer CV specificity nan% +\- nan
%-8s | outer CV precision nan% +\- nan
%-8s | outer CV F1 nan% +\- nan
%-8s | outer CV auc nan +\- nan
Results for CALIFORNIA 0.2 SOFTMAX
%-8s | outer CV accuracy nan% +\- nan
%-8s | outer CV recall nan% +\- nan
%-8s | outer CV specificity nan% +\- nan
%-8s | outer CV precision nan% +\- nan
%-8s | outer CV F1 nan% +\- nan
%-8s | outer CV auc nan +\- nan
*********************
Results for FINLAN

%-8s | outer CV recall 54.85% +\- 26.979
%-8s | outer CV specificity 75.00% +\- 14.907
%-8s | outer CV precision 68.98% +\- 22.437
%-8s | outer CV F1 57.76% +\- 21.158
%-8s | outer CV auc 0.65 +\- 0.140
Results for ALL 0.4 SOFTMAX
%-8s | outer CV accuracy 69.64% +\- 15.017
%-8s | outer CV recall 64.85% +\- 26.411
%-8s | outer CV specificity 75.00% +\- 14.907
%-8s | outer CV precision 70.48% +\- 22.162
%-8s | outer CV F1 65.69% +\- 22.280
%-8s | outer CV auc 0.70 +\- 0.152
*********************
Results for CALIFORNIA 0.4 SVM
%-8s | outer CV accuracy nan% +\- nan
%-8s | outer CV recall nan% +\- nan
%-8s | outer CV specificity nan% +\- nan
%-8s | outer CV precision nan% +\- nan
%-8s | outer CV F1 nan% +\- nan
%-8s | outer CV auc nan +\- nan
Results for CALIFORNIA 0.4 SOFTMAX
%-8s | outer CV accuracy nan% +\- nan
%-8s | outer CV recall nan% +\- nan
%-8s | outer CV specificity nan% +\- nan
%-8s | outer CV precision nan% +\- nan
%-8s | outer CV F1 nan% +\- nan
%-8s | outer CV auc nan +\- nan

%-8s | outer CV F1 55.72% +\- 19.137
%-8s | outer CV auc 0.62 +\- 0.124
Results for ALL 0.63 SOFTMAX
%-8s | outer CV accuracy 65.40% +\- 10.637
%-8s | outer CV recall 61.52% +\- 25.968
%-8s | outer CV specificity 70.00% +\- 17.159
%-8s | outer CV precision 67.18% +\- 19.349
%-8s | outer CV F1 61.16% +\- 19.197
%-8s | outer CV auc 0.66 +\- 0.109
*********************
Results for CALIFORNIA 0.63 SVM
%-8s | outer CV accuracy nan% +\- nan
%-8s | outer CV recall nan% +\- nan
%-8s | outer CV specificity nan% +\- nan
%-8s | outer CV precision nan% +\- nan
%-8s | outer CV F1 nan% +\- nan
%-8s | outer CV auc nan +\- nan
Results for CALIFORNIA 0.63 SOFTMAX
%-8s | outer CV accuracy nan% +\- nan
%-8s | outer CV recall nan% +\- nan
%-8s | outer CV specificity nan% +\- nan
%-8s | outer CV precision nan% +\- nan
%-8s | outer CV F1 nan% +\- nan
%-8s | outer CV auc nan +\- nan
*********************
Results for FINLAND 0.63 SVM
%-8s | outer CV accuracy nan% +\- nan
%-8s | outer CV recall nan% +\- nan
%-

%-8s | outer CV recall 51.06% +\- 23.451
%-8s | outer CV specificity 68.33% +\- 27.080
%-8s | outer CV precision 66.85% +\- 19.357
%-8s | outer CV F1 53.10% +\- 15.643
%-8s | outer CV auc 0.60 +\- 0.089
Results for ALL 0.86 SOFTMAX
%-8s | outer CV accuracy 64.60% +\- 10.951
%-8s | outer CV recall 57.88% +\- 22.659
%-8s | outer CV specificity 71.67% +\- 23.333
%-8s | outer CV precision 69.45% +\- 18.559
%-8s | outer CV F1 59.83% +\- 18.005
%-8s | outer CV auc 0.65 +\- 0.111
*********************
Results for CALIFORNIA 0.86 SVM
%-8s | outer CV accuracy nan% +\- nan
%-8s | outer CV recall nan% +\- nan
%-8s | outer CV specificity nan% +\- nan
%-8s | outer CV precision nan% +\- nan
%-8s | outer CV F1 nan% +\- nan
%-8s | outer CV auc nan +\- nan
Results for CALIFORNIA 0.86 SOFTMAX
%-8s | outer CV accuracy nan% +\- nan
%-8s | outer CV recall nan% +\- nan
%-8s | outer CV specificity nan% +\- nan
%-8s | outer CV precision nan% +\- nan
%-8s | outer CV F1 nan% +\- nan
%-8s | outer CV auc nan +\- 

In [28]:
cv_acc_models = {metric: {name: {'mean': [], 'SD': []} for name, gs_est in sorted(gridcvs.items())} for metric in metrics}

for k in list_of_k:
    for name, gs_est in sorted(gridcvs.items()):
        for metric in metrics:
            cv_acc_models[metric][name]['mean'].append(mean_std_cv_scores[k]['all'][name][metric]['mean'])
            cv_acc_models[metric][name]['SD'].append(mean_std_cv_scores[k]['all'][name][metric]['SD'])
cv_acc_models

{'accuracy': {'SVM': {'mean': [0.6623188405797101,
    0.6456521739130435,
    0.6699275362318841,
    0.6615942028985506,
    0.6525362318840578,
    0.6358695652173912,
    0.661231884057971,
    0.661231884057971,
    0.6608695652173913,
    0.6775362318840579,
    0.6608695652173913,
    0.6608695652173913,
    0.7043478260869565,
    0.6615942028985508,
    0.6876811594202898,
    0.6460144927536232,
    0.6373188405797101,
    0.6623188405797101,
    0.653623188405797,
    0.653623188405797,
    0.653623188405797,
    0.6706521739130435,
    0.6706521739130435,
    0.6706521739130435,
    0.6623188405797101,
    0.6619565217391304,
    0.678623188405797,
    0.678623188405797,
    0.6619565217391304,
    0.6369565217391304,
    0.6369565217391304,
    0.6539855072463767,
    0.6365942028985507,
    0.6452898550724637,
    0.6369565217391304,
    0.6369565217391304,
    0.6369565217391304,
    0.6452898550724637,
    0.6452898550724637,
    0.653623188405797,
    0.645289855072463

In [29]:
for name, gs_est in sorted(gridcvs.items()):
    plt.figure()
    #plt.scatter(list_of_k, cv_acc_models['accuracy'][name]['mean'])
    plt.errorbar(x=list_of_k, y=cv_acc_models['accuracy'][name]['mean'], yerr=cv_acc_models['accuracy'][name]['SD'], linestyle='None', marker='o')
    plt.xlabel('number of features')
    plt.ylabel('cv accuracy')
    plt.title('cv accuracy for ' + name)
    plt.show()

1   HIToolbox                           0x00007ff8171a6726 _ZN15MenuBarInstance22EnsureAutoShowObserverEv + 102
2   HIToolbox                           0x00007ff8171a62b8 _ZN15MenuBarInstance14EnableAutoShowEv + 52
3   HIToolbox                           0x00007ff817115cd7 _ZN15MenuBarInstance21UpdateAggregateUIModeE21MenuBarAnimationStylehhh + 1113
4   HIToolbox                           0x00007ff8171a6173 _ZN15MenuBarInstance19SetFullScreenUIModeEjj + 175
5   AppKit                              0x00007ff81066d4b7 -[NSApplication _setPresentationOptions:instance:flags:] + 1145
6   AppKit                              0x00007ff8104c2165 -[NSApplication _updateFullScreenPresentationOptionsForInstance:] + 582
7   CoreFoundation                      0x00007ff80d2a16d6 __CFNOTIFICATIONCENTER_IS_CALLING_OUT_TO_AN_OBSERVER__ + 137
8   CoreFoundation                      0x00007ff80d33acbc ___CFXRegistrationPost_block_invoke + 86
9   CoreFoundation                      0x00007ff80d33ac13 _CFXR

1   HIToolbox                           0x00007ff8171330c2 _ZN15MenuBarInstance22RemoveAutoShowObserverEv + 30
2   HIToolbox                           0x00007ff8171a6f72 _ZN15MenuBarInstance15DisableAutoShowEv + 24
3   HIToolbox                           0x00007ff8171a7077 _ZN15MenuBarInstanceD2Ev + 113
4   HIToolbox                           0x00007ff8171a6ebb _ZN15MenuBarInstance7ReleaseEv + 35
5   AppKit                              0x00007ff810aa8a0d -[NSHIPresentationInstance discard] + 277
6   AppKit                              0x00007ff810f10952 -[_NSFullScreenSpace(PresentationInstance) discardPresentationInstance] + 36
7   AppKit                              0x00007ff810f109c7 -[_NSFullScreenSpace(PresentationInstance) activateFullScreenPresentationOptions] + 86
8   AppKit                              0x00007ff810d28db9 -[_NSExitFullScreenTransitionController _doSucceededToExitFullScreen] + 38
9   AppKit                              0x00007ff810d2991b __63-[_NSExitFullScreenT

In [38]:
confedence_interval = {k: {center_name: {name: {metric: () for metric in metrics} for name, gs_est in gridcvs.items()} for center_name in center_names} for k in list_of_k}

dof = 5-1 
confidence = 0.95

for k in list_of_k:
    for center_name in center_names:
        for name, gs_est in sorted(gridcvs.items()):
            for metric in metrics:

                m = mean_std_cv_scores[k][center_name][name][metric]['mean']
                s = mean_std_cv_scores[k][center_name][name][metric]['SD']

                t_crit = np.abs(t.ppf((1-confidence)/2,dof))
                confedence_interval[k][center_name][name][metric]= ((m-s*t_crit/np.sqrt(5))*100, (m+s*t_crit/np.sqrt(5))*100)
            
confedence_interval

{0.0: {'all': {'Softmax': {'accuracy': (52.64686285075093, 83.22270236664038),
    'recall': (40.19946455887542, 89.19447483506397),
    'specificity': (52.56522271839265, 90.4650803119104),
    'precision': (51.30447603702179, 88.486288753743),
    'F1': (47.090611458579005, 84.70934297295439),
    'auc': (52.7921584044734, 83.41996280764783)},
   'SVM': {'accuracy': (49.118997565321784, 83.34477055062024),
    'recall': (26.71652691423506, 96.31377611606796),
    'specificity': (54.978883393896204, 88.05141963640682),
    'precision': (44.146473315454905, 88.28942412044255),
    'F1': (34.671898118333154, 88.34397489753985),
    'auc': (49.31222831387243, 83.71807471643062)}},
  'california': {'Softmax': {'accuracy': (nan, nan),
    'recall': (nan, nan),
    'specificity': (nan, nan),
    'precision': (nan, nan),
    'F1': (nan, nan),
    'auc': (nan, nan)},
   'SVM': {'accuracy': (nan, nan),
    'recall': (nan, nan),
    'specificity': (nan, nan),
    'precision': (nan, nan),
    'F

In [ ]:
merged_features_ids = {k: {center_name: [] for center_name in center_names} for k in list_of_k}
merged_features_names = {k: {center_name: [] for center_name in center_names} for k in list_of_k}

for k in list_of_k:
    
    for center_name in center_names:
    
        #print(center_name)
        for i in range(len(features_ids[k][center_name])):
            #print(i)
            length = len(features_ids[k][center_name][i])
            #print(length)
            for j in range(length):
                merged_features_ids[k][center_name].append(features_ids[k][center_name][i][j])
                merged_features_names[k][center_name].append(features_names[k][center_name][i][j])

merged_features_ids

In [ ]:
for k in list_of_k:
    for center_name in center_names: 
        print('Number of features: ' + str(k))
        print(len(merged_features_ids[k][center_name]))

In [ ]:
import operator

counts = {k: {center_name: {} for center_name in center_names} for k in list_of_k}

for k in list_of_k: 
    print('Number of features: ' + str(k))
    for center_name in center_names:
        print(center_name)
        
        for feature_name in merged_features_names[k][center_name]:
            #print(feature_name)
            count = merged_features_names[k][center_name].count(feature_name)
            #print(count)
            counts[k][center_name].update({feature_name: count})

        counts[k][center_name] = sorted(counts[k][center_name].items(),key=operator.itemgetter(1),reverse=True)
        print(len(counts[k][center_name]))
        
print(counts)

In [ ]:
for k in list_of_k:
    for center_name in center_names:
        print(center_name)
        print('Number of features: ' + str(k))
        merged_features_ids[k][center_name] = list(set(merged_features_ids[k][center_name]))
        print(len(merged_features_ids[k][center_name]))